Tracks out the consecutive sequences (the windows in which all of the track ids in one frame is bijective to the track ids in the next frame).

To filter by agent type car, set variable FILTER_AGENT_TYPES as True

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
folderpath = "*"
file = folderpath + "\\lyftlong\\rand_agents_table0.csv"
at0 = pd.read_csv(file)
file = folderpath + "\\lyftlong\\rand_frames_table1.csv"
ft0 = pd.read_csv(file)
file = folderpath +  "\\lyftlong\\rand_scenes_table1.csv"
st0 = pd.read_csv(file)

In [ ]:
at = at0
ft = ft0
st = st0

In [ ]:
# preliminary Agent table cleaning

In [ ]:
at = at.merge(ft[['frame_index', 'scene_index']], on='frame_index')

In [ ]:
def filter_agent_types(at):
    KNOWN_LABELS = ['PERCEPTION_LABEL_CAR', 'PERCEPTION_LABEL_PEDESTRIAN', 'PERCEPTION_LABEL_CYCLIST', 'PERCEPTION_LABEL_UNKNOWN']
    atkl = at[KNOWN_LABELS]

    atklarr = atkl.to_numpy()
    for x in range(len(atkl)):
        row = atklarr[x]
        new_row = np.zeros((4,))
        new_row[np.argmax(row)] = 1
        atklarr[x] = new_row

    # rounds all predictions in known labels to be either 1 or 0, since some are decimals

    for label, column in zip(KNOWN_LABELS, atklarr.T):
        at[label] = column

    at = at[at.PERCEPTION_LABEL_CAR == 1]
    return at
FILTER_AGENT_TYPES = True

if FILTER_AGENT_TYPES:
    at = filter_agent_types(at)
else:
    at = at

In [ ]:
at_scenes = pd.unique(at['scene_index'])
at_scenes

In [ ]:
si = 10

In [ ]:
at2 = at[['track_id', 'frame_index', 'centroid_x', 'scene_index']]
at2 = at2.sort_values(['frame_index', 'track_id'])
at2

In [ ]:
def windows(scene_index: int):
    ats1 = at2[at2.scene_index == scene_index]
    track_ids_set = list(ats1.groupby('frame_index')['track_id'].apply(list))
    return track_ids_set
track_ids_set = windows(7827)
track_ids_set

In [ ]:
def consecutive_windows(track_ids_set):
    intervals = []
    interval = []
    for i in range(1, len(track_ids_set)):
        f1 = track_ids_set[i]
        f0 = track_ids_set[i - 1]
        if set(f1) == set(f0): # if overlap occurs
            interval.append(i - 1)
        else: # if no overlap
            if interval == []: # if interval is empty, carry on
                pass
            else: # if interval isn't empty, we have found end of interval 
                interval.append(i - 1) # close it off
                intervals.append(interval) # put the interval in the intervals stack
                interval = [] # clear the interval
    return intervals
def partial_consecutive_windows(track_ids_set):
    intervals = []
    interval = []
    for i in range(1, len(track_ids_set)):
        f1 = track_ids_set[i]
        f0 = track_ids_set[i - 1]
        if set(f1) == set(f0): # if overlap occurs
            interval.append(i - 1)
        else: # if no overlap
            if interval == []: # if interval is empty, carry on
                pass
            else: # if interval isn't empty, we have found end of interval 
                interval.append(i - 1) # close it off
                intervals.append(interval) # put the interval in the intervals stack
                interval = [] # clear the interval
    return intervals

In [ ]:
all_windows = []
for i in range(0, len(at_scenes)):
    x = list(at_scenes)[i]
    if i%1 == 0:
        print("{0} of {1}".format(i, 100))
        try:
            f = open("lyft_window1.txt", "x")
            f.write(str(all_windows))
            f.close()
        except:
            f = open("lyft_window1.txt", "w")
            f.write(str(all_windows))
            f.close()            
    track_ids_set = windows(x)
    window = consecutive_windows(track_ids_set)
    all_windows.append(window)

In [ ]:
all_windows

In [ ]:
windows_unpacked = [i for x in all_windows for i in x]

In [ ]:
lengths = [len(seq) for seq in windows_unpacked]

In [ ]:
np.median(lengths)

In [ ]:
np.sum(lengths)

In [ ]:
len(lengths)